In [1]:
project_dir = os.path.split(os.getcwd())[0]
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import numpy as np
from IPDL.utils import moving_average as mva
from IPDL import InformationPlane_

In [3]:
a = np.array([[1,2,3,4,5]])
mva(a, n=3)

array([2., 3., 4.])

In [4]:
ip = InformationPlane_.InformationPlane()

In [5]:
ip.Ixt = np.array([[1,2,3,4,5]])
ip.Ity = np.array([[1,2,3,4,5]])

In [6]:
ip.getMutualInformation(moving_average_n=2)

([array([1.5, 2.5, 3.5, 4.5])], [array([1.5, 2.5, 3.5, 4.5])])